# Work with Oracle

In [1]:
import cx_Oracle
import oracledb
import pandas as pd
import numpy as np

In [ ]:
# print(f'\ncx_Oracle.version :{cx_Oracle.version}')
# print("Client version:", cx_Oracle.clientversion())

In [ ]:
# newconn = oracledb.connect(user="<...>", 
#                            password="<...>", 
#                            dsn="<...>/<...>")

In [3]:
conStr = 'NARUT_E01044451/G0to@wlb#mDB001@TDM-scan.dc2.true.th:1521/TDMDBPR'
conn = cx_Oracle.connect(conStr)
print('TDMDBPR : Connected')
cur = conn.cursor()

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help

## Sample

In [ ]:
# print(f'\ncx_Oracle.version :{cx_Oracle.version}')
# # print("Client version:", cx_Oracle.clientversion())

# # cx_Oracle.init_oracle_client(lib_dir="C:\Oracle\Ora11\product\11.2.0.4\client_1")
# # cx_Oracle.init_oracle_client(lib_dir="C:\Instantclient\instantclient_21_10")

# # AIMD2CDB
# # conStr = 'NARUT_E01044451/v9VzR8xsuwp#tyTb@TDM-scan.dc2.true.th:1521/TDMDBPR'
# conStr = 'AUTOKPI/SNRpAssw0rd#001@172.16.54.52:1555/AKPIPRD'
# conn = cx_Oracle.connect(conStr)
# cur = conn.cursor()

# # sqlTxt = "SELECT * FROM AUTOKPI.TARGET_BILL_CYCLE_DAILY WHERE TM_KEY_DAY = 20230501"
# # cur.execute(sqlTxt)
# # sqlTxt = "SELECT * FROM AUTOKPI.TARGET_BILL_CYCLE_DAILY WHERE TM_KEY_DAY =:1"
# # cur.execute(sqlTxt, ('20230501',))
# sqlTxt = "SELECT * FROM AUTOKPI.DIM_MOOC_AREA WHERE ROWNUM <= :1"
# cur.execute(sqlTxt, ('100',))

# # print(f'\ndesctiption :\n{cur.description}')

# rows = cur.fetchall()
# # print(f'\nrows :\n{rows}')
# # print(f'rows :', cur.rowcount)
# # TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, DAYS_IN_MONTH, PERIODFLAG, METRIC_CD, METRIC_NAME, METRIC_GRP, METRIC_VALUE, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK

# df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])
# print(f'\ndf :\n{df}')

# cur.close()
# conn.close()

## 6 Dimension Summary

In [ ]:
conStr = 'NARUT_E01044451/v9VzR8xsuwp#tyTb@TDM-scan.dc2.true.th:1521/TDMDBPR'
conn = cx_Oracle.connect(conStr)
print('TDMDBPR : Connected')
cur = conn.cursor()

v_week = (2023022, )


try:
    # print("Database version:", conn.version)
    print(f'\nExecute query as Week {v_week[0]}...')

    sqlTxt = """
        WITH W_VAR (AS_OF_WK) AS 
        ( 
            SELECT :1 AS AS_OF_WK FROM DUAL 
        ) 
        , W_TMP_PERIOD (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, WK_DAY_START, WK_DAY_END) AS 
        ( 
            SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, MIN(TM_KEY_DAY) AS WK_DAY_START, MAX(TM_KEY_DAY) AS WK_DAY_END 
            FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK 
            WHERE TM_KEY_WK <= (SELECT AS_OF_WK FROM W_VAR) 
            GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK 
        ) 
        SELECT * 
        FROM ( 
            SELECT TM_KEY_MTH, TM_KEY_WK, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME, DIMENSION_KPI_FLAG, BG_FLAG--, METRIC_NAME_GROUP 
                , AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD 
                , CAST(SUM(ACTUAL_AGG) AS DECIMAL(18,2)) AS ACTUAL_AGG 
                , CAST(SUM(TARGET_AGG) AS DECIMAL(18,2)) AS TARGET_AGG 
                , MAX(PPN_TM) AS PPN_TM 
            FROM ( 
                SELECT TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME 
                    , CASE 	WHEN ( METRIC_CD IN ('DNEC10006', 'TNEC10002', 'TNEC20002') --NETWORK EXPERIENCE 
                                OR METRIC_CD IN ('B1R000100', 'B2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00142') --BUSINESS GROWTH 
                                OR METRIC_CD IN ('DSER21101', 'DSER22101', 'TSER21101', 'TSER22101', 'TSER23101', 'DB2S000200', 'DB1S000200', 'TB2S000200', 'TB1S000200', 'TSER13100', 'TSER14100') --CUSTOMER RETENTION & EXPERIENCE 
                                OR METRIC_CD IN ('TNSC00011', 'TNSC00013', 'TNSC00087', 'TNSC00101', 'TNSC00144', 'TNSC00020', 'TNSC00130') --NEW S-CURVE 
                                --COST & PRODUCTIVITY (Unknown ?) 
                                OR METRIC_CD IN ('DIS10007', 'DIS10011', 'DIS10001') --DIGITALIZATION 
                            ) THEN 'Y' ELSE 'N' END DIMENSION_KPI_FLAG 
                    , CASE 	WHEN METRIC_CD IN ('B1R000100', 'B2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00142') --BUSINESS GROWTH 
                            THEN 'Y' ELSE 'N' END BG_FLAG 
                    , METRIC_NAME_GROUP, PPN_TM, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_AGG, TARGET_AGG 
                FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK 
                WHERE TM_KEY_DAY IN (SELECT WK_DAY_END FROM W_TMP_PERIOD) 
                AND AREA_TYPE = 'P' 
            ) A 
            GROUP BY TM_KEY_MTH, TM_KEY_WK, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME, DIMENSION_KPI_FLAG, BG_FLAG, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD 
        ) B 
        --ORDER BY TM_KEY_MTH, TM_KEY_WK, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD 
    """
    
    cur.execute(sqlTxt, v_week)
    # cur.execute(sqlTxt, ('2023020',))
    # cur.executemany(sqlTxt, multiple parameter)

    print(f'\nDone !!!')

    rows = cur.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])


except cx_Oracle.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print('\nTDMDBPR : Disconnected')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.head()

## Generate Output file

In [ ]:
# C:\Users\Narut4\coding\Jupyter\Work with Oracle.ipynb

output_name = f'Raw-Agg Perf_Wk {v_week[0]}'

df.to_csv(f'{output_name}.csv', index=False, encoding='utf-8')

print(f'\n"{output_name}.csv" is generated')